### setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers

### tokenization

In [ ]:
# useful functions for data cleansing

import unicodedata

def decode_bytes(text):

  '''check for bytes within text and decode them'''

  if isinstance(text, str):
    return text
  elif isinstance(text, bytes):
    return text.decode("utf-8", "ignore")
  else:
    raise ValueError(f"unsupported string type: {text}")

def whitespace_tokenize(text):

  '''strip text and split them'''

  text = text.strip()
  if not text:
    return []
  tokens = text.split()
  return tokens

def is_control(char):

  ''' detect control characters except whitespace '''

  if unicodedata.category(char).startswith('C'):
    if char == '\t' or char == '\n' or char == '\r': # category starts with C but is whitespace
      return False
    return True
  else:
    return False

def is_whitespace(char):

  ''' check whether character is whitespace '''

  if char == ' ' or char == '\t' or char == '\n' or char == '\r':
    return True
  elif unicodedata.category(char) == "Zs": # "space separator" category
    return True
  else:
    return False

def is_punc(char):

  ''' check whether character is a punctuation '''
  # ~32 : 각종 공백문자, 48~57 : 숫자, 65~90 : 대문자 알파벳, 97~122 : 소문자 알파벳, 126~ : 이상한 기호들
  if (ord(char) >= 33 and ord(char) <= 47) or (ord(char) >= 58 and ord(char) <= 64):
    return True
  elif (ord(char) >= 91 and ord(char) <= 96) or (ord(char) > 123 and ord(char) <= 126):
    return True
  elif unicodedata.category(char).startswith("P"): # punctuation
    return True
  else:
    return False

In [ ]:
import collections

def load_vocab(vocab_file):

  ''' load vocab file as dictionary '''

  vocab_dict = collections.OrderedDict()
  idx=0

  with open(vocab_file, "r") as rf:
    tokens = rf.read().splitlines()

  for token in tokens:
    token = decode_bytes(token)
    token = token.strip()
    vocab_dict[token] = idx
    idx += 1

  return vocab_dict

In [ ]:
class BasicTokenizer(object):
  def __init__(self, do_lower_case=True):
    self.do_lower_case = do_lower_case

  def clean_text(self, text):

    ''' skip invalid characters, convert all whitespaces into single space, and return text '''

    output = []
    for char in text:
      # check if char is NULL,�(unrecognizable), or control character
      if ord(char) == 0 or ord(char) == 65533 or is_control(char):
        continue
      if is_whitespace(char):
        output.append(" ") # all whitespace -> " "
      else:
        output.append(char)
    return "".join(output)

  def run_split_on_punc(self, text):

    ''' split text based on punctuations '''

    output = []
    token = ''
    for char in list(text):
      if is_punc(char):
        output.append(token)
        output.append(char)
        token = ''
      else:
        token += char

    if len(token) > 0:
      output.append(token)

    return output

  def tokenize(self, text):
    text = decode_bytes(text)
    text = self.clean_text(text)

    orig_tokens = whitespace_tokenize(text) # 공백문자 단위로 분리
    split_tokens = []

    for token in orig_tokens: # 토큰별로 punctuation 있을 경우 분리
      if self.do_lower_case:
        token = token.lower()
      split_tokens.extend(self.run_split_on_func(token))

    return split_tokens

In [ ]:
class WordpieceTokenizer(object):
  def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=200):
    self.vocab = vocab
    self.unk_token = unk_token
    self.max_input_chars_per_word = max_input_chars_per_word

  def tokenize(self, text):
    ''' input : a single token (via BasicTokenizer), output : wordpiece tokens

    wordpiece merges characters by "score = freq(pair) / freq(first)*freq(second)".
    pairs that frequently appear together are merged, but not if each element also appears frequently

    ex) unable : "un", "##able"
    ex) hugging : "hugging" '''

    text = decode_bytes(text)

    output_tokens = []
    for token in whitespace_tokenize(text):
      chars = list(token)
      # 너무 긴 단어 (>200)는 unk 처리
      if len(chars) > self.max_input_chars_per_word:
        output_tokens.append(self.unk_token)
        continue

      is_bad = False
      start = 0
      sub_tokens = []

      # ex. unfriendly : length 10
      while start < len(chars): # 0~9
        end = len(chars) # 10
        cur_substr = None
        while start < end:
          substr = "".join(chars[start:end]) # unfriendly, unfriendl, unfriend, ... , un
          if start > 0:
            substr = "##" + substr # indicates subword
          if substr in self.vocab: # "unfriendly" not in vocab / ... "un" in vocab
            cur_substr = substr
            break
          end -= 1
        if cur_substr is None:
          is_bad = True
          break
        sub_tokens.append(cur_substr)

        start = end # subword 떨어져나간 경우 (un) 끝자리->시작자리 (f)

      if is_bad: # character 한개도 vocab에 없는 경우
        output_tokens.append(self.unk_token)
      else:
        output_tokens.extend(sub_tokens)

    return output_tokens



In [ ]:
class FullTokenizer(object):
  def __init__(self, vocab_file, do_lower_case=True):
    self.vocab = load_vocab(vocab_file)
    self.vocab_reverse = {v:k for k,v in self.vocab.items()}
    self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case)
    self.wordpiece = WordpieceTokenizer(vocab=self.vocab)

  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text): # cleanse text, tokenize by punctuation
      for sub_token in self.wordpiece.tokenize(token): # tokenize tokens (subtokens)
        split_tokens.append(sub_token)

    return split_tokens

### finetune biobert

In [ ]:
from absl import flags

flags.DEFINE_string("bert_config_file", None, "config json file corresponding to the pretrained BERT model")
flags.DEFINE_string("vocab_File", None, "vocab file on which BERT was trained")
flags.DEFINE_string("output_dir", None, "output dir for model checkpoints")
flags.DEFINE_string("train_file", None, "squad-formatted json for training. E.g., train-v1.1.json")
flags.DEFINE_string("predict_file", None, "squad-formatted json for predictions. E.g., dev(test)-v1.1.json")
flags.DEFINE_string("init_checkpoint", None, "initial checkpoint (usually from a pretrained BERT model)")

flags.DEFINE_bool("do_lower_case", True, "whether to lower-case input text. True for lower case.")
flags.DEFINE_bool("do_train", True, "whether to run training")
flags.DEFINE_bool("do_predict", True, "whether to run eval on the dev set")
flags.DEFINE_bool("use_tpu", False, "whether to use TPU or GPU/CPU")
flags.DEFINE_bool("verbose_logging", False, "whether to print all warnings during data processing. Other warnings are printed by default.")

flags.DEFINE_integer("max_seq_len", 384, "maximum input sequence after WordPiece tokenization. Will be padded if shorter, truncated if longer.")
flags.DEFINE_integer("doc_stride", 128, "when splitting up a long document into chunks, how much stride to take between chunks")
flags.DEFINE_integer("max_query_len", 64, "maximum query sequence after WordPiece tokenization. Will be truncated if longer.")
flags.DEFINE_integer("predict_batch_size", 8, "total batch size for predictions")
flags.DEFINE_integer("save_checkpoint_step", 1000, "how often to save model checkpoints")
flags.DEFINE_integer("iterations_per_loop", 1000, "how many steps to make in each estimator call")
flags.DEFINE_integer("n_best_size", 20, "how many n-best predictions to generate in nbest_predictions.json output file")
flags.DEFINE_integer("max_answer_len", 30, "maximum length of generated answer")

flags.DEFINE_float("learning_rate", 5e-5, "initial learning rate for Adam optimizer")
flags.DEFINE_float("num_train_epochs", 3.0, "number of training epochs")
flags.DEFINE_float("warmup_proportion", 0.1, "proportion of training for linear lr warmup. E.g., 0.1 refers to 10% of training")
flags.DEFINE_float("null_score_diff_threshold", 0.0, "to predict null if (null_score - best_non_null) > threshold")

In [ ]:
# import sys
# sys.path.insert(1, '/content/drive/MyDrive/biobert')

In [ ]:
class SquadExample(object):
  def __init__(self, qas_id, question_text, doc_tokens, origin_answer_text=None, start_pos=None, end_pos=None, is_impossible=False):
    self.qas_id = qas_id
    self.question_text = question_text
    self.doc_tokens = doc_tokens
    self.origin_answer_text = origin_answer_text
    self.start_pos = start_pos
    self.end_pos = end_pos
    self.is_impossible = is_impossible

  def __str__(self):
    return self.__repr__()

  def __repr__(self):
    s = f"qas_id: {decode_bytes(self.qas_id)}" # check if text is in str, bytes or unicode and convert
    s += f", question_text: {decode_bytes(self.question_text)}"
    s += f", doc_tokens: {[' '.join(self.doc_tokens)]}"

    if self.start_pos:
      s += f", start_pos: {self.start_pos}, end_pos: {self.end_pos}, is_impossible: {self.is_impossible}"

    return s


In [ ]:
import json
import tensorflow as tf

class InputFeatures(object):
  def __init(self, unique_id, example_index, doc_span_index, tokens, token_to_origin_map, token_is_max_context,
             input_ids, input_mask, segment_ids, start_pos=None, end_pos=None, is_impossible=None):
    self.unique_id = unique_id
    self.example_index = example_index
    self.doc_span_index = doc_span_index
    self.tokens = tokens
    self.token_to_origin_map = token_to_origin_map
    self.token_is_max_context = token_is_max_context
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.start_pos = start_pos
    self.end_pos = end_pos
    self.is_impossible = is_impossible

In [ ]:
def read_squad_examples(input_file, is_training):
  """Read a squad json file into a list of SquadExample"""

  with open(input_file, "r") as rf:
    input_data = json.load(rf)["data"][0]["paragraphs"]

  examples = []
  for entry in input_data:
    doc_tokens = []
    char_to_word_offset = []
    prev_is_whitespace = True

    context = entry["context"]
    for c in context:
      if is_whitespace(c):
        prev_is_whitespace=True
      else:
        if prev_is_whitespace: # c comes after whitespace -> add as new token
          doc_tokens.append(c)
        else:
          doc_tokens[-1] += c # c comes after c -> attach to recent token
        prev_is_whitespace=False
      char_to_word_offset.append(len(doc_tokens) -1) # "I went home" -> [0,0,1,1,1,1,1,2,2,2,2]

    qas = entry["qas"]
    for qa in qas:
      qas_id = qa["id"]
      question_text = qa["question"]
      start_pos = None
      end_pos = None
      origin_answer_text = None
      is_impossible = False

      if is_training:
        if len(qa["answers"]) != 1:
          raise ValueError("Each question should have exactly 1 answer.")

        answer = qa["answers"][0]
        origin_answer_text = answer["text"] #"Bazex syndrome"
        answer_offset = answer["answer_start"] #93
        answer_length = len(origin_answer_text)
        start_pos = char_to_word_offset[answer_offset] # 몇번째 단어부터
        end_pos = char_to_word_offset[answer_offset + answer_length-1] # 몇번째 단어까지

        # check if answer text is extractable from context (아닌 경우 건너뜀)
        answer_from_context = " ".join(doc_tokens[start_pos:end_pos+1]) # context에서 주어진 범위로 인덱싱한 정답
        cleaned_answer = " ".join(whitespace_tokenize(origin_answer_text)) # 실제 정답 (strip 및 split)

        if answer_from_context.find(cleaned_answer) == -1: # context에 정답이 들어있지 않은 경우
          tf.logging.warning(f"Could not find answer from context. {answer_from_context} vs {cleaned_answer}")
          continue

      else: # inference용
        start_pos = -1
        end_pos = -1
        origin_answer_text = ""

      example = SquadExample(qas_id=qas_id, question_text=question_text, doc_tokens=doc_tokens,
                              origin_answer_text=origin_answer_text, start_pos=start_pos, end_pos=end_pos,
                              is_impossible=is_impossible)
      examples.append(example)

  return examples

In [ ]:
examples = read_squad_examples('/content/drive/MyDrive/cose474_final/trainset_bioasq.json', is_training=True)

In [ ]:
print(examples[0])
print(examples[0].question_text) # each element in "read_squad_examples" is a SquadExample object.
print(type(examples[0]))

qas_id: 56bc751eac7ad10019000013_001, question_text: Name synonym of Acrokeratosis paraneoplastica., doc_tokens: ['Psoriasiform dermatitis in a case of newly diagnosed locally advanced pyriform sinus tumour: Bazex syndrome revisited. Acrokeratosis paraneoplastica of Bazex is a rare but important paraneoplastic dermatosis, usually manifesting as psoriasiform rashes over the acral sites. It often precedes diagnosis of the associated malignancy, usually that of upper aerodigestive tract squamous cell carcinoma. We present the case of a patient with a newly diagnosed pyriform sinus tumour and associated acrokeratosis paraneoplastica. To the best of our knowledge, this is the first reported case in the local literature.'], start_pos: 13, end_pos: 14, is_impossible: False
Name synonym of Acrokeratosis paraneoplastica.
<class '__main__.SquadExample'>


In [ ]:
def convert_examples_to_features(examples, tokenizer, max_seq_len, doc_stride, max_query_len, is_training, output_fn):
  unique_id = 1000000000

  for idx, example in enumerate(examples):
    query_tokens = tokenizer.tokenize(example.question_text)

    if len(query_tokens) > max_query_len: # cut query if it exceeds maximum token length
      query_tokens = query_tokens[:max_query_len]

    token_to_orig_idx = []
    orig_to_token_idx = []
    all_doc_tokens = []

    for i, token in enumerate(example.doc_tokens): # whitespace로만 분리된 tokens
      orig_to_token_idx.append(len(all_doc_tokens))
      sub_tokens = tokenizer.tokenize(token) # tokenize into subwords
      for sub_token in sub_tokens:
        token_to_orig_idx.append(i)
        all_doc_tokens.append(sub_token)